In [3]:
import sys
import os
sys.path.append(os.path.abspath("../src"))

In [4]:
import networkx as nx
import dwave_networkx as dnx
from dwave.system import DWaveSampler
from dwave.system import AutoEmbeddingComposite
%matplotlib inline
import matplotlib.pyplot as plt
from random import choice
import numpy as np
import itertools
import os
import pandas as pd
import pickle
from tqdm.notebook import tqdm
import re
from utils.basic import func_max, QUBO_bias, generate_qubo_value
from utils.show_results import print_func, extract_numbers
from utils.calculation import generate_empty_dataframe, find_intermediate_solutions

In [2]:
sampler =  AutoEmbeddingComposite(DWaveSampler())

# Soluzioni intermedie complete

In [9]:
# DO NOT RUN
folder = "./output/static_ratio_more_batch"
folder_for = f"{folder}/graphs"
times = pd.DataFrame()
for file_name in sorted(os.listdir(folder_for), key=extract_numbers):
    #extract values from folder
    graph = f"{folder}/graphs/{file_name}"
    cli_func = f"{folder}/cli_func/{file_name}"
    with open(graph, "rb") as f:
        G = pickle.load(f)
    with open(cli_func, "rb") as f:
        cli_func = pickle.load(f)
    df, cliques = generate_empty_dataframe(cli_func)
    with open(f"{folder}/empty_dataframes/{file_name}", "wb") as f:
        pickle.dump(df, f)
    with open(f"{folder}/empty_dataframes/cliques_{file_name}", "wb") as f:
        pickle.dump(cliques, f)
    times[file_name]=[0]
with open(f"{folder}/times_parziali.pkl", "wb") as f:
    pickle.dump(times, f)

In [9]:
folder = "./output/static_ratio_more_batch"
folder_for = f"{folder}/graphs"
with open(f"{folder}/times_parziali.pkl", "rb") as f:
        times = pickle.load(f)
reads = 500
batch = 8
for file_name in sorted(os.listdir(folder_for), key=extract_numbers):
    #extract values from folder
    print(f"Calculating {file_name}...")
    graph_f = f"{folder}/graphs/{file_name}"
    cli_func_f = f"{folder}/cli_func/{file_name}"
    df_f = f"{folder}/full_dataframes/{file_name}"
    cliques_f = f"{folder}/empty_dataframes/cliques_{file_name}"
    with open(graph_f, "rb") as f:
        G = pickle.load(f)
    with open(cli_func_f, "rb") as f:
        cli_func = pickle.load(f)
    with open(df_f, "rb") as f:
        df = pickle.load(f)
    with open(cliques_f, "rb") as f:
        cliques = pickle.load(f)
    #qubo_value = generate_qubo_value(G, cli_func)
    qubo = QUBO_bias(G, False, True, [1,2,4,8]) #qubo_value per scalare in base alla differenza delle medie
    for _ in tqdm(range(batch)):
        df = find_intermediate_solutions(G, sampler, df, cliques, qubo, reads)
        times[file_name]+=reads
    with open(df_f, "wb") as f:
        pickle.dump(df, f)
  
with open(f"{folder}/times_parziali.pkl", "wb") as f:
    pickle.dump(times, f)   

Calculating 9_21.pkl...


  0%|          | 0/8 [00:00<?, ?it/s]

Calculating 12_28.pkl...


  0%|          | 0/8 [00:00<?, ?it/s]

Calculating 15_35.pkl...


...

  0%|          | 0/8 [00:00<?, ?it/s]

Calculating 42_98.pkl...


  0%|          | 0/8 [00:00<?, ?it/s]

Calculating 45_105.pkl...


  0%|          | 0/8 [00:00<?, ?it/s]

In [10]:
times

,9_21.pkl,12_28.pkl,15_35.pkl,18_42.pkl,21_49.pkl,24_56.pkl,27_63.pkl,30_70.pkl,33_77.pkl,36_84.pkl,39_91.pkl,42_98.pkl,45_105.pkl
0,8000,8000,8000,8000,8000,8000,8000,8000,8000,8000,8000,8000,8000


In [22]:
folder = "./output/static_ratio_2"
folder_for = f"{folder}/graphs"
for file_name in sorted(os.listdir(folder_for), key=extract_numbers):
    graph_f = f"{folder}/graphs/{file_name}"
    cli_func_f = f"{folder}/cli_func/{file_name}"
    df_f = f"{folder}/full_dataframes/{file_name}"
    cliques_f = f"{folder}/empty_dataframes/cliques_{file_name}"
    with open(df_f, "rb") as f:
        df = pickle.load(f)
    print(df)

   0   1   2   3   4   5   6   7   8   9   10  11  12  13   14   15    16
0   0   0   0   0   0   0   0   0   1   0  -1   0   0   0    0    0     0
1   0   0   0   0   0   0   0   0   0   0   0   1   0  26  270  429  3273
2   0   0   0   0   0   0  -1   0   0   0   0   0   0   0    0    0     0
   0   1   2   3   4   5   6   7   8   ...  10  11  12  13  14  15   16   17    18
0   0   0   0   0   0   0   0   0   0  ...   0  11   6  -1   0   0    0    0     0
1   0   0   0   0   0   0   0   0   0  ...   0   0   0   0   2  13  175  235  3551
2   0   0   0   0   0   0   0   0   3  ...   0   0   0   0   0   0    0    0     0
3   0   0   0  -1   0   0   0   0   0  ...   0   0   0   0   0   0    0    0     0

[4 rows x 19 columns]
   0   1   2   3   4   5   6   7   8   ...  18  19  20  21  22   23   24   25    26
0   0   0   0   0   0   0   0   0   0  ...   0   0   0   0   0    0    0    0     0
1   0   0   0   0   0   0   0   0   0  ...   0   0   1   7  20  113  445  964  1653
2   0   0   0 